In [1]:
import pandas as pd
import numpy as np
np.random.seed(529)

# データの準備 

In [2]:
# データの読み込み
email_data = pd.read_csv('http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv')

# 女性向けメール配信データの削除と介入変数の追加
male_df = email_data.copy()  # SettingWithCoppyWarningを回避するためにコピーしているが、本質的に意味はない
male_df = male_df[male_df['segment'] != 'Womens E-Mail']
male_df['treatment'] = male_df['segment'].apply(lambda x: 1 if x == 'Mens E-Mail' else 0)

# バイアスのあるデータの準備
obs_rate_c = 0.5
obs_rate_t = 0.5

biased_data = male_df.copy()
biase_rule = (biased_data['history'] > 300) | (biased_data['recency'] < 6) | (biased_data['channel'] == 'Multichannel')
biased_data['obs_rate_c'] = biase_rule.apply(lambda x: obs_rate_c if x else 1)
biased_data['obs_rate_t'] = biase_rule.apply(lambda x: 1 if x else obs_rate_t)
biased_data['random_number'] = np.random.uniform(size=biased_data.shape[0])
biased_data = (
    biased_data[
                ((biased_data['treatment'] == 0) & (biased_data['random_number'] < biased_data['obs_rate_c']))
                | ((biased_data['treatment'] == 1) & (biased_data['random_number'] < biased_data['obs_rate_t']))
                ]
)

# バイアスデータの回帰分析

## sklearn

In [3]:
from sklearn.linear_model import LinearRegression

# モデルの学習
X = biased_data[['treatment', 'history']]
y = biased_data['spend']
model = LinearRegression(fit_intercept=True, normalize=False).fit(X, y)

In [4]:
# 結果の出力
print(f'R^2: {model.score(X, y)}')
print(f'intercept: {model.intercept_}')
print(f'coefficients: {model.coef_}')

R^2: 0.0012301106028368425
intercept: 0.36686204599715955
coefficients: [0.80139138 0.00130696]


## statsmodels

In [5]:
from statsmodels.formula.api import ols

# モデルの学習
model = ols('spend ~ treatment + history', data=biased_data).fit()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [6]:
# 結果の出力
model.summary()  # treatmentのcoef(0.8014)とP値(0.000)に着目する

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  spend   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     19.73
Date:                Wed, 24 Jun 2020   Prob (F-statistic):           2.72e-09
Time:                        13:28:09   Log-Likelihood:            -1.3406e+05
No. Observations:               32050   AIC:                         2.681e+05
Df Residuals:                   32047   BIC:                         2.682e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3669      0.148      2.472      0.013       0.076       0.658
treatment      0.8014      0.179      4.469      0.000       0.450       1.153
history        0.0013      0.000      3.778      0.000       0.001       0.002
==============================================================================
Omnibus:                    70681.398   Durbin-Watson:                   2.003
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        340206674.375
Skew:                          20.568   Prob(JB):                         0.00
Kurtosis:                     506.055   Cond. No.                         832.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# 共変量の与える効果

In [7]:
# RCTデータ
rct_reg = ols('spend ~ treatment', data=male_df).fit()
rct_reg.summary() # 介入効果(0.7698)がRCT比較による結果と一致する

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  spend   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     28.09
Date:                Wed, 24 Jun 2020   Prob (F-statistic):           1.16e-07
Time:                        13:28:09   Log-Likelihood:            -1.7584e+05
No. Observations:               42613   AIC:                         3.517e+05
Df Residuals:                   42611   BIC:                         3.517e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6528      0.103      6.356      0.000       0.451       0.854
treatment      0.7698      0.145      5.300      0.000       0.485       1.055
==============================================================================
Omnibus:                    94877.860   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        502270597.344
Skew:                          21.023   Prob(JB):                         0.00
Kurtosis:                     533.203   Cond. No.                         2.62
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [8]:
# 切片と推定値に関するテーブルのみ指定できる
rct_reg.summary().tables[1]  # treatmentに関する推定値が0.7698

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.6528,0.103,6.356,0.000,0.451,0.854
treatment,0.7698,0.145,5.300,0.000,0.485,1.055


In [9]:
# バイアスデータの単回帰
nonrct_reg = ols('spend ~ treatment', data=biased_data).fit()
nonrct_reg.summary().tables[1] # セレクションバイアスによって、本来の効果よりも過剰に値が推定される(0.7698 -> 0.8921)

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.6356,0.130,4.878,0.000,0.380,0.891
treatment,0.8921,0.178,5.019,0.000,0.544,1.240


In [10]:
# バイアスデータの重回帰
nonrct_mreg = ols('spend ~ treatment + recency + channel + history', data=biased_data).fit()
nonrct_mreg.summary().tables[1] # 結果が本来の値に近づく

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.3705,0.388,0.955,0.340,-0.390,1.131
channel[T.Phone],0.2091,0.311,0.672,0.502,-0.401,0.819
channel[T.Web],0.4646,0.311,1.495,0.135,-0.144,1.074
treatment,0.7414,0.183,4.043,0.000,0.382,1.101
recency,-0.0477,0.027,-1.792,0.073,-0.100,0.004
history,0.0013,0.000,3.466,0.001,0.001,0.002


# 脱落変数バイアスの確認

In [11]:
model_A = ols('spend ~ treatment + recency + channel', data=biased_data).fit()
model_B = ols('spend ~ treatment + recency + channel + history', data=biased_data).fit()
model_C = ols('history ~ treatment + recency + channel', data=biased_data).fit()

In [12]:
model_A.summary().tables[1] # treatment_coef = 0.7794

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,1.1397,0.318,3.582,0.000,0.516,1.763
channel[T.Phone],-0.1953,0.288,-0.677,0.498,-0.761,0.370
channel[T.Web],0.0613,0.288,0.213,0.832,-0.503,0.626
treatment,0.7794,0.183,4.257,0.000,0.421,1.138
recency,-0.0669,0.026,-2.565,0.010,-0.118,-0.016


In [13]:
model_B.summary().tables[1]  #history_coef = 0.0013, treatment_coef=0.7414

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.3705,0.388,0.955,0.340,-0.390,1.131
channel[T.Phone],0.2091,0.311,0.672,0.502,-0.401,0.819
channel[T.Web],0.4646,0.311,1.495,0.135,-0.144,1.074
treatment,0.7414,0.183,4.043,0.000,0.382,1.101
recency,-0.0477,0.027,-1.792,0.073,-0.100,0.004
history,0.0013,0.000,3.466,0.001,0.001,0.002


In [14]:
model_C.summary().tables[1]  # treatmen_coef=28.3990

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,575.0531,4.605,124.866,0.000,566.026,584.080
channel[T.Phone],-302.3718,4.175,-72.419,0.000,-310.556,-294.188
channel[T.Web],-301.5214,4.170,-72.303,0.000,-309.695,-293.348
treatment,28.3990,2.650,10.718,0.000,23.206,33.592
recency,-14.3068,0.377,-37.920,0.000,-15.046,-13.567


In [15]:
print(f'OVB: {model_B.params["history"] * model_C.params["treatment"]}')
print(f'coef_gap: {model_A.params["treatment"] - model_B.params["treatment"]}')

OVB: 0.03798751588492059
coef_gap: 0.03798751588492277


# 入れてはいけない変数の投入

In [16]:
cor_visit_treatment = ols('treatment ~ visit + channel + recency + history', data=biased_data).fit()
cor_visit_treatment.summary().tables[1]  # treatmentとvisitに相関がある

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,0.7085,0.011,63.778,0.000,0.687,0.730
channel[T.Phone],-0.0620,0.009,-6.580,0.000,-0.080,-0.044
channel[T.Web],-0.0652,0.009,-6.932,0.000,-0.084,-0.047
visit,0.1444,0.008,19.068,0.000,0.130,0.159
recency,-0.0286,0.001,-36.097,0.000,-0.030,-0.027
history,0.0001,1.17e-05,9.817,0.000,9.18e-05,0.000


In [17]:
bad_control_reg = ols('spend ~ treatment + recency + channel + history + visit', data=biased_data).fit()
bad_control_reg.summary().tables[1]  # treatmentのcoefが大きく低下する

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,-0.5898,0.384,-1.536,0.125,-1.343,0.163
channel[T.Phone],0.3201,0.307,1.043,0.297,-0.282,0.922
channel[T.Web],0.3366,0.307,1.098,0.272,-0.264,0.938
treatment,0.1696,0.182,0.932,0.351,-0.187,0.526
recency,0.0026,0.026,0.100,0.920,-0.049,0.054
history,0.0008,0.000,2.218,0.027,9.82e-05,0.002
visit,7.3602,0.248,29.672,0.000,6.874,7.846
